In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
import torch
import os

# to disable torchdynamo completely. Faster for smaller systems and single-point calculations.
os.environ["TORCHDYNAMO_DISABLE"] = "1"  # hard-disable capture

import numpy as np
import math
import matplotlib.pyplot as plt
import time

from dftorch.SCF import SCFx, SCFx_batch
from dftorch.DM_Fermi_x import DM_Fermi_x
from dftorch.Constants import Constants
from dftorch.Energy import Energy, EnergyShadow
from dftorch.Forces import forces_shadow, forces_shadow_pme
from dftorch.Tools import calculate_dist_dips
from dftorch.Structure import Structure, StructureBatch
from dftorch.Kernel_Fermi import Kernel_Fermi
from dftorch.MD import initialize_velocities, initialize_velocities_batch, MDXL, MDXLBatch
from dftorch.XLTools import kernel_update_lr, calc_q
from dftorch.io import read_xyz_traj_data, read_xyz, write_XYZ_trajectory
from dftorch.ESDriver import ESDriver, ESDriverBatch
from dftorch.ewald_pme import calculate_PME_ewald, init_PME_data, calculate_alpha_and_num_grids, ewald_energy

from sedacs.neighbor_list import NeighborState

torch._dynamo.config.capture_dynamic_output_shape_ops = True

import warnings
import logging

# Silence warnings and module logs
warnings.filterwarnings("ignore")
os.environ["TORCH_LOGS"] = ""               # disable PT2 logging
os.environ["TORCHINDUCTOR_VERBOSE"] = "0"
os.environ["TORCHDYNAMO_VERBOSE"] = "0"

logging.getLogger("torch.fx").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.symbolic_shapes").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.recording").setLevel(logging.CRITICAL)
logging.getLogger("torch._dynamo").setLevel(logging.CRITICAL)
logging.getLogger("torch._inductor").setLevel(logging.CRITICAL)

torch.set_default_dtype(torch.float64)
torch.cuda.empty_cache()

In [38]:
%%time
dftorch_params = {
    'coul_method': '!PME',
    'Coulomb_acc': 1e-5, # coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 10.0, # coulomb cutoff
    'PME_order': 4,

    'SCF_MAX_ITER': 20,
    'SCF_TOL': 1e-6,
    'SCF_ALPHA': 0.3, # Linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 30,
    'KRYLOV_TOL': 1e-6,
    'KRYLOV_TOL_MD': 1e-4,
    'KRYLOV_START': 3,
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda'

# filename = ['water_30.xyz']*1
# LBox = torch.tensor([[30,30,30]]*1, device=device)

# filename = ['COORD_8WATER.xyz']*2
# LBox = torch.tensor([[20,20,20], [9,9,9]], device=device)

# filename = ['C840.xyz']*3
# LBox = torch.tensor([[33,33,33], [37,37,37], [37,37,37]], device=device)

# filename = ['C840.xyz']*2
# LBox = torch.tensor([[34,34,34]]*2, device=device)

filename = ['COORD.xyz']*2
LBox = torch.tensor([[10,15,15], [9,3,3]], device=device)

# filename = ['C1.xyz', 'C2.xyz']
# LBox = torch.tensor([[30,30,30], [15,30,30]], device=device)

# filename = ['C1.xyz']
# LBox = torch.tensor([[30,30,30]], device=device)

# filename = ['C1.xyz']*4
# LBox = torch.tensor([[20,35,35]]*4, device=device)

const = Constants(
    filename,
    '/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
    #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1',
    param_grad = False).to(device)
Efield = 0*0.3*torch.tensor([-.3,0.4,0.0], device=device).T # In arbitrary direction  Works ony in 0-field!!!
structure = StructureBatch(
    filename, LBox, const, charge=0, e_field=Efield, device=device,
    req_grad_xyz=False)
structure.Te = 3000.0 # Some electronic temperature in Kelvin, Possible bug at high tempertures!!!

es_driver = ESDriverBatch(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure, const, do_scf=True, verbose=True)

es_driver.calc_forces(structure, const)
structure.e_tot - structure.e_repulsion

/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-C.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/C-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/C-C.skf
  t <batched neighbor list> 0.002 s  (B=2, N=20)
H0_and_S
  Do H off-diag
  t <dR and pair mask> 0.0 s

  t <SKF> 0.0 s

  Do H and S
H0_and_S t 0.1 s

  t <batched neighbor list> 0.002 s  (B=2, N=20)
  t <batched neighbor list> 0.002 s  (B=2, N=20)
   LMAX: tensor([3, 3], device='cuda:0', dtype=torch.int32)
   0
   1
   2
   3

Starting cycle
Iter 1
Batch 0: Res = 3.514e-01, dEc = 9.691e+00
Batch 1: Res = 3.458e-01, dEc = 9.627e+00
Iter 2
Batch 0: Res = 1.955e-01, dEc = 4.947e-02
Batch 1: Res = 2.026e-01, dEc = 5.177e-02
Iter 3
Batch 0: Res = 1.091e-01, dEc = 2.584e-02
Batch 1: Res = 1.189e-01, dEc = 2.826e-02
Iter 4
  rank: 0, batch 0, Fel = 0.000662
  rank: 0, batch 1, Fel = 0.000629
  Not co

tensor([-339.8958, -330.4569], device='cuda:0')

In [37]:
torch.manual_seed(0)
temperature_K = torch.tensor([20.0, 30.0], device=structure.device)
mdDriver = MDXLBatch(es_driver, const, temperature_K=temperature_K)
mdDriver.run(structure, dftorch_params, num_steps=60, dt=0.3)

########## Step = 0 ##########
   LMAX: tensor([3, 3], device='cuda:0', dtype=torch.int32)
H0: 0.275 s
H1: 0.004 s
  rank: 0, batch 0, Fel = 0.000243
  rank: 0, batch 1, Fel = 0.000491
  Not converged: 2
  rank: 1, batch 0, Fel = 0.000026
  rank: 1, batch 1, Fel = 0.000065
  Not converged: 0
KER: 0.006 s
F AND E: 0.002 s
ETOT = -329.71785481, EPOT = -329.95321454, EKIN = 0.04653360, T = 17.99997718, ResErr = 0.000000, t = 0.3 s
ETOT = -320.25571938, EPOT = -320.49091422, EKIN = 0.06980039, T = 26.99996577, ResErr = 0.000000, t = 0.3 s
0.01164032 GB


########## Step = 1 ##########
   LMAX: tensor([3, 3], device='cuda:0', dtype=torch.int32)
H0: 0.250 s
H1: 0.004 s
  rank: 0, batch 0, Fel = 0.000300
  rank: 0, batch 1, Fel = 0.000434
  Not converged: 2
  rank: 1, batch 0, Fel = 0.000041
  rank: 1, batch 1, Fel = 0.000091
  Not converged: 0
KER: 0.006 s
F AND E: 0.002 s
ETOT = -329.71911965, EPOT = -330.45714513, EKIN = 0.23409489, T = 90.55183762, ResErr = 0.001657, t = 0.3 s
ETOT = -320

In [ ]:
%%time
dftorch_params = {
    'coul_method': '!PME',
    'Coulomb_acc': 1e-5, # coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 10.0, # coulomb cutoff
    'PME_order': 4,

    'SCF_MAX_ITER': 20,
    'SCF_TOL': 1e-6,
    'SCF_ALPHA': 0.3, # Linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 30,
    'KRYLOV_TOL': 1e-6,
    'KRYLOV_TOL_MD': 1e-4,
    'KRYLOV_START': 3,
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda'

# filename = 'C840.xyz'
# LBox = torch.tensor([33,33,33], device=device)

# filename = 'COORD.xyz'
# LBox = torch.tensor([9,3,3], device=device)

filename = 'COORD_8WATER.xyz'
LBox = torch.tensor([20,20,20], device=device)

# filename = 'water_30.xyz'
# LBox = torch.tensor([30,30,30], device=device)

# filename = 'COORD.xyz'
# LBox = torch.tensor([9,4,4], device=device)

# filename = 'C1.xyz'
# LBox = torch.tensor([30,30,30], device=device)

# filename = 'C2.xyz'
# LBox = torch.tensor([15,30,30], device=device)

const = Constants(
    filename,
    '/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
    #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1',
    param_grad = False).to(device)
Efield = 0*0.3*torch.tensor([-.3,0.4,0.0], device=device).T # In arbitrary direction  Works ony in 0-field!!!

structure1 = Structure(
    filename, LBox, const, charge=0, e_field=Efield, device=device,
    req_grad_xyz=False)
structure1.Te = 3000.0 # Some electronic temperature in Kelvin, Possible bug at high tempertures!!!

es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const)

structure1.e_tot - structure1.e_repulsion

/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-O.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-O.skf
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

### Do SCF ###
  Initial DM_Fermi

Starting cycle
Iter 1
Res = 1.473967917, dEc = 7.726816588, t = 0.0 s

Iter 2
Res = 0.779885694, dEc = 1.463443100, t = 0.0 s

Iter 3
Res = 0.415319004, dEc = 0.716727998, t = 0.0 s

Iter 4
  rank: 0, Fel = 0.002371
  rank: 1, Fel = 0.000263
  rank: 2, Fel = 0.000023
  rank: 3, Fel = 0.000002
  rank: 4, Fel = 0.000000
Res = 0.222174972, dEc = 0.773845792, t = 0.0 s

Iter 5
  rank: 0, Fel = 0.000003
  rank: 1, Fel = 0.000000
Res = 0.000261210, dEc = 0.000855060, t = 0.0 s

Iter 6
  rank: 0, Fel = 0.000000
Res = 0.000000773, dEc = 0.000000267, t = 0.0 s

CPU times: user 3.83 s, sys: 6

In [7]:
torch.manual_seed(0)
temperature_K = torch.tensor(300.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
mdDriver.run(structure1, dftorch_params, num_steps=40, dt=0.3)

########## Step = 0 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

H0: 0.434 s
H1: 0.002 s
  rank: 0, Fel = 0.000447
  rank: 1, Fel = 0.000034
KER: 0.003 s
F AND E: 0.002 s
ETOT = -872.94996648, EPOT = -873.95685062, EKIN = 0.85311593, T = 274.99965138, ResErr = 0.000000, t = 0.4 s
0.020121088 GB


########## Step = 1 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

H0: 0.416 s
H1: 0.003 s
  rank: 0, Fel = 0.000250
  rank: 1, Fel = 0.000027
KER: 0.003 s
F AND E: 0.002 s
ETOT = -872.95456956, EPOT = -874.27317422, EKIN = 1.00228106, T = 323.08263687, ResErr = 0.002024, t = 0.4 s
0.0201216 GB


########## Step = 2 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

H0: 0.425 s
H1: 0.003 s
  rank: 0, Fel = 0.000317
  rank: 1, Fel = 0.000035
KER: 0.003 s
F AND E: 0.002 s
ETOT = -872.96188839, EPOT = -874.63371253, EKIN = 1.31128582, T = 422.68950046, ResErr = 0.00132